In [68]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

pd.set_option("max_columns", 500)
pd.set_option('display.float_format', lambda x: "{:,.4f}".format(x))

In [69]:
df = pd.read_csv("../data/winequality-red.csv", sep=";")

In [71]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4000,0.7000,0.0000,1.9000,0.0760,11.0000,34.0000,0.9978,3.5100,0.5600,9.4000,5
1,7.8000,0.8800,0.0000,2.6000,0.0980,25.0000,67.0000,0.9968,3.2000,0.6800,9.8000,5
2,7.8000,0.7600,0.0400,2.3000,0.0920,15.0000,54.0000,0.9970,3.2600,0.6500,9.8000,5
3,11.2000,0.2800,0.5600,1.9000,0.0750,17.0000,60.0000,0.9980,3.1600,0.5800,9.8000,6
4,7.4000,0.7000,0.0000,1.9000,0.0760,11.0000,34.0000,0.9978,3.5100,0.5600,9.4000,5


In [70]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,"1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000","1,599.0000"
mean,8.3196,0.5278,0.2710,2.5388,0.0875,15.8749,46.4678,0.9967,3.3111,0.6581,10.4230,5.6360
std,1.7411,0.1791,0.1948,1.4099,0.0471,10.4602,32.8953,0.0019,0.1544,0.1695,1.0657,0.8076
min,4.6000,0.1200,0.0000,0.9000,0.0120,1.0000,6.0000,0.9901,2.7400,0.3300,8.4000,3.0000
25%,7.1000,0.3900,0.0900,1.9000,0.0700,7.0000,22.0000,0.9956,3.2100,0.5500,9.5000,5.0000
50%,7.9000,0.5200,0.2600,2.2000,0.0790,14.0000,38.0000,0.9968,3.3100,0.6200,10.2000,6.0000
75%,9.2000,0.6400,0.4200,2.6000,0.0900,21.0000,62.0000,0.9978,3.4000,0.7300,11.1000,6.0000
max,15.9000,1.5800,1.0000,15.5000,0.6110,72.0000,289.0000,1.0037,4.0100,2.0000,14.9000,8.0000


In [72]:
X = df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 
        'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol',]]
y = df[["quality"]]

### Escalamiento

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [74]:
mms_x = MinMaxScaler()
mms_y = MinMaxScaler()
Xs = mms_x.fit_transform(X_train)
ys = mms_y.fit_transform(y_train)

In [75]:
pd.DataFrame(ys)

,0
0,0.4000
1,0.8000
2,0.4000
3,1.0000
4,0.6000
...,...
1194,0.6000
1195,0.6000
1196,0.4000
1197,0.4000


## Modelado 

### Regresión

#### Sin escalamiento

#### Con escalamiento

In [76]:
sgdr = SGDRegressor()
sgdr.fit(X_train, y_train)
ls_medias = cross_val_score(estimator=sgdr, X=X_train, y=y_train, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)

/home/jorge/Documents/Diplomado/ambientes/diplo_env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


(-4.011768584200453e+24, 3.7407286121141147e+24)

In [77]:
sgdr = SGDRegressor()
sgdr.fit(Xs, ys)
ls_medias = cross_val_score(estimator=sgdr, X=Xs, y=ys, cv = 4, n_jobs=-1, scoring="r2")
np.mean(ls_medias), np.std(ls_medias)


/home/jorge/Documents/Diplomado/ambientes/diplo_env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


(0.13775902797547374, 0.012499295935014611)

In [80]:
SGDRegressor?

In [81]:
param_grid = {"loss": ["squared_loss", "huber"],
              "penalty": ["l2", "l1", "elasticnet"],
              "alpha": [x/100 for x in range(100)],
              "eta0": [.1,.001,.0001,.4],
              "learning_rate": ["constant", "invscaling", "adaptive"],
             }

In [82]:
n_hyper = np.prod(list(map(len, param_grid.values())))

In [83]:
n_hyper

7200

In [43]:
model = SGDRegressor(alpha=0.01)
hyper = GridSearchCV(estimator=model, n_jobs=-1, scoring="r2", param_grid=param_grid, error_score=-1000, verbose=5)
hyper.fit(Xs, ys)
hyper.best_score_

Fitting 5 folds for each of 7200 candidates, totalling 36000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1864 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 5896 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 11080 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 17416 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 24904 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 33544 tasks      | elapsed:   38.4s
[Parallel(n_jobs=-1)]: Done 36000 out of 36000 | elapsed:   41.4s finished
/home/jorge/Documents/Diplomado/ambientes/diplo_env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.3449310586697679

In [44]:
hyper.best_estimator_

SGDRegressor(alpha=0.0, eta0=0.1, learning_rate='adaptive')

In [45]:
mms_y.inverse_transform([hyper.best_estimator_.intercept_])

array([[5.75716195]])

#### Hyperparametrización

##### Sin escalamiento

##### Con escalamiento

In [46]:
%%time
model = SGDRegressor()
hyper = RandomizedSearchCV(estimator=model, n_jobs=-1, scoring="r2", param_distributions=param_grid, error_score=-1000, verbose=1, n_iter=int(n_hyper*.25), random_state=777)
hyper.fit(X_train, y_train)
hyper.best_score_

Fitting 5 folds for each of 1800 candidates, totalling 9000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1800 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 5800 tasks      | elapsed:   20.7s


CPU times: user 10.8 s, sys: 116 ms, total: 10.9 s
Wall time: 29.7 s


[Parallel(n_jobs=-1)]: Done 9000 out of 9000 | elapsed:   29.6s finished
/home/jorge/Documents/Diplomado/ambientes/diplo_env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.2894364630721356

In [47]:
%%time
model = SGDRegressor()
hyper = RandomizedSearchCV(estimator=model, n_jobs=-1, scoring="r2", param_distributions=param_grid, error_score=-1000, verbose=1, n_iter=int(n_hyper*.25), random_state=777)
hyper.fit(Xs, ys)
hyper.best_score_

Fitting 5 folds for each of 1800 candidates, totalling 9000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 3824 tasks      | elapsed:    6.1s


CPU times: user 9.38 s, sys: 120 ms, total: 9.5 s
Wall time: 12.9 s


[Parallel(n_jobs=-1)]: Done 9000 out of 9000 | elapsed:   12.8s finished
/home/jorge/Documents/Diplomado/ambientes/diplo_env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.34419307479820205

In [48]:
X_train.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')

In [49]:
hyper.best_estimator_.coef_

array([ 0.05135016, -0.30108365, -0.02036286,  0.11180294, -0.26081174,
        0.04968504, -0.21021519, -0.09266913, -0.06281409,  0.35214008,
        0.31725174])

### Clasificación 

In [84]:
y_train_c = (y_train >= y_train.mean())*1

In [85]:
y_test_c = (y_test >= y_train.mean())*1

In [86]:
y_train_c

,quality
977,0
243,1
688,0
1403,1
232,1
...,...
405,1
1158,1
456,0
864,0


In [87]:
sgdc = SGDClassifier()
sgdc.fit(X_train, y_train_c)
ls_medias = cross_val_score(estimator=sgdc, X=X_train, y = y_train_c, cv = 4, n_jobs=-1, scoring="roc_auc")
np.mean(ls_medias), np.std(ls_medias)

/home/jorge/Documents/Diplomado/ambientes/diplo_env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


(0.6761036269329587, 0.09805636039853506)

In [53]:
SGDClassifier?

In [88]:
param_grid = {"loss": ["hinge", "log", "modified_huber", "squared_hinge", "perceptron"],
              "penalty": ["l2", "l1", "elasticnet"],
              "alpha": [x/100+0.01 for x in range(100)],
              "l1_ratio": [x/10 for x in range(10)],
              "learning_rate": ["constant", "optimal", "invscaling", "adaptive"],
              "eta0": [0.001,.01,.5,.00001]
             }

In [89]:
n_hyper = np.prod(list(map(len, param_grid.values())))

In [90]:
n_hyper

240000

#### Hyperparametrización

##### Sin escalamiento

##### Con escalamiento

In [57]:
%%time
model = SGDClassifier()
hyper = RandomizedSearchCV(estimator=model, n_jobs=-1, scoring="roc_auc", param_distributions=param_grid, error_score=-1000, verbose=1, n_iter=int(n_hyper*.1), random_state=777, cv = 4)
hyper.fit(X_train, y_train_c)
hyper.best_score_

Fitting 4 folds for each of 24000 candidates, totalling 96000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1104 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 3104 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 5904 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 9504 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done 13904 tasks      | elapsed:   54.7s
[Parallel(n_jobs=-1)]: Done 19104 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 25104 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 31904 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 39504 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 47904 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 57104 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 67104 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 77904 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 89504 

CPU times: user 3min 3s, sys: 2.45 s, total: 3min 5s
Wall time: 7min 32s


/home/jorge/Documents/Diplomado/ambientes/diplo_env/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.8005960736069531

In [58]:
%%time
model = SGDClassifier()
hyper = RandomizedSearchCV(estimator=model, n_jobs=-1, scoring="roc_auc", param_distributions=param_grid, error_score=-1000, verbose=1, n_iter=int(n_hyper*.1), random_state=777, cv=4)
hyper.fit(Xs, y_train_c.values.ravel())
hyper.best_score_

Fitting 4 folds for each of 24000 candidates, totalling 96000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 2160 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 6160 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 11760 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 18960 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done 27760 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 38160 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 50160 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 63760 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 78960 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 95760 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 96000 out of 96000 | elapsed:  3.2min finished


CPU times: user 2min 14s, sys: 1.7 s, total: 2min 15s
Wall time: 3min 11s


0.8134435673510881

In [59]:
hyper.best_estimator_.coef_

array([[ 0.00093973, -0.00164159, -0.00064508,  0.00065021, -0.00129728,
         0.00048913, -0.00397147, -0.00045614,  0.00046105,  0.00511071,
         0.00502576]])

In [60]:
model_best=hyper.best_estimator_

In [61]:
from sklearn.metrics import classification_report

In [63]:
print(classification_report(y_train_c, model_best.predict(X_train)))

              precision    recall  f1-score   support

           0       0.49      0.87      0.63       557
           1       0.66      0.22      0.33       642

    accuracy                           0.52      1199
   macro avg       0.57      0.54      0.48      1199
weighted avg       0.58      0.52      0.47      1199



In [67]:
print(classification_report(y_test_c, model_best.predict(X_test)))

              precision    recall  f1-score   support

           0       0.48      0.85      0.61       187
           1       0.58      0.18      0.28       213

    accuracy                           0.49       400
   macro avg       0.53      0.52      0.45       400
weighted avg       0.53      0.49      0.43       400

